# Multiclass SVM 구현

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn import preprocessing

#IRIS 데이터 로드
iris =  sns.load_dataset('iris') 
X= iris.iloc[:,:4] #학습할데이터
y = iris.iloc[:,-1] #타겟
print(y)

0         setosa
1         setosa
2         setosa
3         setosa
4         setosa
         ...    
145    virginica
146    virginica
147    virginica
148    virginica
149    virginica
Name: species, Length: 150, dtype: object


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=48)

In [5]:
X_train

,sepal_length,sepal_width,petal_length,petal_width
110,6.5,3.2,5.1,2.0
69,5.6,2.5,3.9,1.1
148,6.2,3.4,5.4,2.3
39,5.1,3.4,1.5,0.2
53,5.5,2.3,4.0,1.3
...,...,...,...,...
64,5.6,2.9,3.6,1.3
91,6.1,3.0,4.6,1.4
81,5.5,2.4,3.7,1.0
51,6.4,3.2,4.5,1.5


In [6]:
def standardization(train, test):
    scaler = StandardScaler()
    train = scaler.fit_transform(train)
    test = scaler.transform(test)
    return train, test

X_train, X_test = standardization(X_train, X_test)

In [7]:
X_train # 스케일링 적용한 후

array([[ 0.78522493,  0.32015325,  0.77221097,  1.04726529],
       [-0.26563371, -1.29989934,  0.0982814 , -0.11996537],
       [ 0.43493872,  0.78302542,  0.94069336,  1.43634218],
       [-0.84944407,  0.78302542, -1.24957775, -1.28719604],
       [-0.38239578, -1.7627715 ,  0.15444219,  0.13941922],
       [ 0.55170079, -0.374155  ,  1.05301496,  0.7878807 ],
       [ 0.31817664, -0.14271892,  0.65988937,  0.7878807 ],
       [ 0.20141457, -0.374155  ,  0.43524618,  0.39880381],
       [-1.66677857, -0.14271892, -1.36189934, -1.28719604],
       [-0.14887164, -0.60559109,  0.21060299,  0.13941922],
       [-0.14887164, -1.06846325, -0.12636179, -0.24965767],
       [ 0.31817664, -0.60559109,  0.15444219,  0.13941922],
       [ 0.66846286, -0.83702717,  0.88453256,  0.91757299],
       [ 0.0846525 , -0.14271892,  0.77221097,  0.7878807 ],
       [-0.49915786, -0.14271892,  0.43524618,  0.39880381],
       [-0.26563371, -0.60559109,  0.65988937,  1.04726529],
       [ 2.18636979,  1.

In [8]:
X_test

array([[-0.14887164, -0.374155  ,  0.26676379,  0.13941922],
       [ 0.31817664, -0.60559109,  0.54756778,  0.00972692],
       [ 0.31817664, -1.06846325,  1.05301496,  0.26911151],
       [-1.5500165 , -1.7627715 , -1.36189934, -1.15750374],
       [ 0.0846525 ,  0.32015325,  0.60372857,  0.7878807 ],
       [ 0.78522493, -0.14271892,  0.99685416,  0.7878807 ],
       [-0.84944407,  1.70876975, -1.24957775, -1.15750374],
       [ 0.20141457, -0.14271892,  0.60372857,  0.7878807 ],
       [-0.38239578,  2.63451409, -1.30573855, -1.28719604],
       [-0.38239578, -1.29989934,  0.15444219,  0.13941922],
       [ 0.66846286,  0.08871717,  0.99685416,  0.7878807 ],
       [-0.38239578,  1.0144615 , -1.36189934, -1.28719604],
       [-0.49915786,  0.78302542, -1.13725615, -1.28719604],
       [ 0.43493872, -0.60559109,  0.60372857,  0.7878807 ],
       [ 0.55170079, -1.7627715 ,  0.37908538,  0.13941922],
       [ 0.55170079,  0.55158933,  0.54756778,  0.52849611],
       [-1.19973028,  0.

## Multiclass svm?

기본적으로 SVM은 이진 분류기.

하지만 이진 분류법을 확장해서 멀티 클래스를 분류하는 방법이 있는데, 대표적으로 One vs. One approach // One vs. Rest approach(One vs. all)가 있다.

**1. One vs. One approach**
M개의 클래스가 있다고 가정할 때, M개의 클래스 중 2개를 골라 두 클래스에 대한 초평면을 만든다.(즉, 두 클래스를 나누는 기준이 되는 경계를 만든다.)<br>그러면 총 $_{M}C_{2}$ 개의 초평면과 분류기가 생성된다.

test할 때는 새로 들어온 sample $x$에 대하여 초평면 $d_{ij}(x)$가 x를 클래스 i로 분류하면 클래스 i에 1점을 주고 클래스 j로 분류하면 클래스 j에 1점을 준다. <br> 따라서, 총 $_{M}C_{2}$개의 분류기가 새로 들어온 sample $x$를 분류하게 되고 각 클래스에 점수를 줌으로써, 가장 높은 점수를 받은 클래스가 sample $x$에 대해 예측한 클래스가 된다.

클래스가 개수가 많아지면 분류기의 개수가 많아져 학습에 소요되는 시간이 길어진다. 

> 초평면(Hyperplane): 여러 데이터를 나누는 기준이 되는 경계

In [10]:
pd.get_dummies(y_train)

,setosa,versicolor,virginica
110,0,0,1
69,0,1,0
148,0,0,1
39,1,0,0
53,0,1,0
...,...,...,...
64,0,1,0
91,0,1,0
81,0,1,0
51,0,1,0


In [55]:
class OneVsRestApproach:
    def __init__(self, class_num):
        self.class_num = class_num
        self.model_list = list()
        self.pred = list()

    def one_vs_rest_labels(self, y_train): # one-hot encoding
        y_train = pd.get_dummies(y_train)
        return y_train

    def fit(self, X_train, y_train, gamma):
        y_train2 = pd.get_dummies(y_train)

        for i in range(self.class_num):
            model = SVC(kernel = 'rbf', gamma = gamma)
            model.fit(X_train, y_train2.iloc[:, i]) # 각 클래스에 대하여 각각 fit
            self.model_list.append(model) # fit한 모델을 일단 저장

    def predict(self, X_test):
        size = X_test.shape[0]

        for row in range(size):
            argmax_list = list()
            for model in self.model_list:
                argmax_list.append(model.decision_function(X_test)[row])
            self.pred.append(np.argmax(argmax_list))
        
        self.pred = pd.DataFrame(self.pred).replace({0:'setosa', 1:'versicolor', 2:'virginica'}) # One hot encoding 한 것들 다시 변환
        
        return self.predict

    def evaluate(self, y_test):
        return accuracy_score(y_test, self.pred)

In [60]:
model2 = OneVsRestApproach(class_num=3)

In [61]:
model2.fit(X_train, y_train, gamma = 0.01)

In [62]:
y_pred = model2.predict(X_test)

In [63]:
model2.evaluate(y_test)

0.7333333333333333

### 번외 OneVsOneApproach

마지막 경우의 수를 생각해야 하지만,, 시간 관계상 구현하지 못했습니다.

In [ ]:
class OneVsOneApproach:
    # 생성자 
    def __init__(self, class_num):
        self.class_num = class_num
        self.each_result = list()

    # test를 위해 두 클래스에 대한 라벨을 정리해준다.
    # y_train에 대해 클래스 i, j로 분류한다.
    def labelEncode(self, i, j, y_train):
        labels = list()
        for label in y_train:
            if label == i: # y_train의 label이 i클래스에 속하면 labels 리스트에 1삽입.
                labels.append(1)
            else:
                labels.append(0)
        return np.array(labels) # 라벨이 부여된 array를 반환한다.

    def fit(self, X_train, y_train, gamma): # gamma를 조절하면 하나의 변수가 미치는 영향의 범위를 조절할 수 있다.
        le = preprocessing.LabelEncoder()

        y_train = le.fit_transform(pd.DataFrame(y_train)) # setosa: 0, versicolor: 1, virginica: 2
        for i in range(self.class_num):
            for j in range(i+1, self.class_num):
                # 두 클래스에 해당하는 데이터들만 가져오기
                X_c = X_train[(y_train == i) | (y_train == j)]
                y_c = y_train[(y_train == i) | (y_train == j)]

                model = SVC(kernel = 'rbf', gamma=gamma)
                model.fit(X_c, y_c)
                self.each_result.append(([model, i, j])) # 훈련된 모델과 각 클래스를 리스트에 저장
    
    def predict(self, X_test): # 모든 분류기에 대하여 투표
        size = X_test.shape[0]
        score = np.zeros((size, self.class_num), dtype = int) # 분류를 예측한 리스트
        test_label = list()
        for row in range(size):
            x = X_test[row, :] # 각 row sample에 대해 예측
            for classifier, class1, class2 in self.each_result:
                predict = classifier.predict(x)

                if predict == 1: # classifier가 class1로 분류하면
                    score[row][class1] += 1
                else:
                    score[row][class2] += 1
            
            test_label.append(np.argmax(size[row]))

